<a href="https://colab.research.google.com/github/TheNaughtyGuy/Twitter-Sentiment-Analysis/blob/main/twitterSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import required lib

In [ ]:
import requests
import os
import json
import re
from textblob import TextBlob
import matplotlib.pyplot as plt


#data input from user

In [ ]:
print("Enter the topic for sentiment analysis: ")
topic=str(input())

covid


#Authentication & function for rules

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #printf(response.json())
    return response.json()

In [ ]:
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    #printf(response.json())

In [ ]:
def set_rules(headers, delete, bearer_token):

    Value="#"+topic
    ### Edit the Value according to the mode of sentiment analysis

    ##CORE --- for projects
    #from:particular_user 
    #to:particular_user
    #url:"for particular url"
    #retweets_of:any_username
    #bio:"data engineer"
    #is:verified	
    #lang:en ##hi
  
    ##ADVANCED ---Available for  Academic Research Project.
    #bio_location:delhi
    #place:"new york city"
    #place_country:IN
    #point_radius:[longitude latitude radius] ex-point_radius:[2.355128 48.861118 16km]
    #bounding_box:[west_long south_lat east_long north_lat] #in particular location boundary

    sample_rules = [
        {   
            "value": Value,
            "tag":  Value

        },
    ]
    payload = {"add": sample_rules}
    response = requests.post(

        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #printf(response.json())

In [ ]:
def get_stream(headers, set, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i=0
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            printf(json_response)

#Save dataset in file

In [ ]:
def printf(jsonn):
    out_txt=topic + ".txt";
    out_file = open(out_txt, "a") 
    out_file.write(str(jsonn.get("data").get("text"))+"\n")
    out_file.close() 

#Data Cleansing 

In [ ]:
def clean_tweets():
    in_txt=topic + ".txt";
    in_file = open(in_txt, "r") 
    Lines = in_file.readlines()
    in_file.close()
    out_txt=topic + "_clean.txt";
    out_file = open(out_txt, "a") 
    for tweet in Lines:
      out_file.write(' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())+"\n")
    out_file.close() 


#Sentiment of tweet using TextBlob
#### textBlob is a python lib used for NLP for sentiment analysis...

In [ ]:
def get_tweet_sentiment(tweet):
		analysis = TextBlob(tweet)
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

In [ ]:
def get_output_count():
  in_txt=topic + "_clean.txt";
  in_file = open(in_txt, "r") 
  Lines = in_file.readlines()
  in_file.close()
  n=len(Lines)
  Positive=0
  Neutral=0
  Negative=0
  for tweet in Lines:
      sentiment=get_tweet_sentiment(tweet)
      if sentiment=='positive':
        Positive=Positive+1
      if sentiment=='negative':
        Negative=Negative+1
      if sentiment=='neutral':
        Neutral=Neutral+1
  pos=Positive*100/n
  neg=Negative*100/n
  neu=Neutral*100/n
  return [neg,neu,pos]


#Execution

In [ ]:
def main():
    #bearer_token = os.environ.get("BEARER_TOKEN")
    bearer_token="AAAAAAAAAAAAAAAAAAAAACiKQQEAAAAATwPNnUFydIJqV4G0GONBSh1wCOA%3D03nPjvXKmPHPOwYOmWkNEjvrxPCEnVQ77rnOkw2q6kZ5HK7odq"
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)

    ##0-get_stream
    ##1-data cleansing 
    ##2-output presentation
    print("Enter 0 for get_stream")
    print("Enter 1 for data cleansing")
    print("Enter 2 for output presentation")
    i=int(input())
    if i==0:
      get_stream(headers, set, bearer_token)
    else:
      if i==1:
        clean_tweets()
      else:
        if i==2:
          fig = plt.figure()
          ax = fig.add_axes([0,0,1,1])
          Sentiment = ['Negative','Neutral','Positive']
          percentages = get_output_count()
          ax.bar(Sentiment,percentages)
          plt.show()
        else:
          print("wrong input")


In [ ]:
if __name__ == "__main__":
    main()

0
200
